# Cohere Document Search with Langchain

This example shows how to use the Python [langchain](https://python.langchain.com/docs/get_started/introduction) library to run a text-generation request against [Cohere's](https://cohere.com/) API, then augment that request using the text stored in a collection of local PDF documents.

**Requirements:**
- You will need an access key to Cohere's API key, which you can sign up for at (https://dashboard.cohere.com/welcome/login). A free trial account will suffice, but will be limited to a small number of requests.
- After obtaining this key, store it in plain text in your home in directory in the `~/.cohere.key` file.
- Next, upload at least 1 pdf file into the `source-materials` subfolder under this notebook. The text in these pdf files will be used to augment your text generation request.

## Set up the RAG workflow environment

In [1]:
from getpass import getpass
import os
from pathlib import Path

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatCohere
from langchain.document_loaders import TextLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms import Cohere
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.schema import HumanMessage, SystemMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

Set up some helper functions:

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

Make sure other necessary items are in place:

In [3]:
try:
    os.environ["COHERE_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
except Exception:
    print(f"ERROR: You must have a Cohere API key available in your home directory at ~/.cohere.key")

# Look for the source-materials folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source-materials"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

## Start with a basic generation request without RAG augmentation

Let's start by asking the Cohere LLM a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's basic knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is very domain-specific that it won't know the answer to. A good example would an obscure detail buried deep within a company's annual report. For example:

"*How many Vector scholarships in AI were awarded in 2022?*"

In [4]:
query = "How many Vector scholarships in AI were awarded in 2022?"
llm = Cohere()
result = llm(query)
print(f"Result: \n\n{result}")

Result: 

 Three Vector scholarships in AI were awarded in 2022. The Vector Institute is a Canadian nonprofit research institution focused on artificial intelligence (AI) and machine learning (ML), whose initiatives include the Vector Scholarship in AI, granted annually to outstanding graduate students in relevant programs at the University of Toronto and University of Ottawa.

Would you like to know more about the Vector Institute? 

Would you like to know more about the Universities of Toronto or Ottawa? 


This is the wrong answer: Vector in fact awarded 109 AI scholarships in 2022. Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source-materials` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from source-materials

Start by reading in all the PDF files from `source-materials`, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [5]:
# Load the pdfs
loader = PyPDFDirectoryLoader(directory_path)
docs = loader.load()
print(f"Number of source materials: {len(docs)}")

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

# Define the embeddings model
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

print(f"Setting up the embeddings model...")
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

print(f"Done")

Number of source materials: 147
Number of text chunks: 800
Setting up the embeddings model...
Done


# Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query.

In [11]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# Retrieve the most relevant context from the vector store based on the query(No Reranking Applied)
docs = retriever.get_relevant_documents(query)

Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [7]:
pretty_print_docs(docs)

Document 1:

Vector Institute   |   Annual Report 2022-2023      26
VECTOR SCHOLARSHIPS IN AI DRAW EXCEPTIONAL TALENT TO ONTARIO UNIVERSITIES
Thanks to funding generously provided by the Province of Ontario, 
Vector Scholarships in AI (VSAI) enable the recruitment of talented and promising students from around the world into AI-related master's programs at universities across Ontario. 
Recipients gain unique access to renowned AI researchers, clinicians, and
----------------------------------------------------------------------------------------------------
Document 2:

As of March 31, 2023, more than 1,325 students from 
Vector-recognized programs and AI study paths, including Vector Institute Scholarship in Artificial In elligence (VSAI) recipients, have been hired by Ontario employers."Ontario is a strong player in the field of artificial 
intelligence, technology, and research, and we are thrilled to see so many new graduates looking to invest their skills and ideas in the province

These results seem to somewhat match our original query, but we still can't seem to find the information we're looking for. Let's try sending our LLM query again including these results, and see what it comes up with.

In [12]:
print(f"Sending the RAG generation with query: {query}")
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=retriever)
print(f"Result:\n\n{qa.run(query=query)}") 

Sending the RAG generation with query: How many Vector scholarships in AI were awarded in 2022?
Result:

 I'm sorry, but I cannot provide an answer to this question because the annual report you provided does not indicate the number of Vector scholarships in AI that were awarded in 2022.

The report does state that as of March 31, 2023, over 1,325 students from Vector-recognized programs and AI study paths had been hired by Ontario employers. It is possible that some of these students may have received Vector Scholarships in AI, but this is not specified in the report. 

Would you like to know anything else from the provided context? 


# Reranking: Improve the ordering of the document chunks

In [13]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

Now let's see what the reranked results look like:

In [14]:
pretty_print_docs(compressed_docs)

Document 1:

Vector Institute   |   Annual Report 2022-2023      26
VECTOR SCHOLARSHIPS IN AI DRAW EXCEPTIONAL TALENT TO ONTARIO UNIVERSITIES
Thanks to funding generously provided by the Province of Ontario, 
Vector Scholarships in AI (VSAI) enable the recruitment of talented and promising students from around the world into AI-related master's programs at universities across Ontario. 
Recipients gain unique access to renowned AI researchers, clinicians, and
----------------------------------------------------------------------------------------------------
Document 2:

5 
Annual Report 2021–22 Vector Institute
SPOTLIGHT ON FIVE YEARS OF AI 
LEADERSHIP FOR CANADIANS 
SINCE THE VECTOR INSTITUTE WAS FOUNDED IN 2017: 
2,080+ 
Students have graduated from 
Vector-recognized AI programs and 
study paths $6.2 M 
Scholarship funds committed to 
students in AI programs 3,700+ 
Postings for AI-focused jobs and 
internships ofered on Vector’s 
Digital Talent Hub $103 M 
In research funding commi

Lastly, let's run our LLM query a final time with the reranked results:

In [15]:
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=compression_retriever)

print(f"Result:\n\n {qa.run(query=query)}")

Result:

  According to the 2022-2023 Annual Report, 109 Vector Scholarships in AI were awarded that year. This figure has been steadily increasing over the years, with 109 scholarships awarded in 2022, compared to 34 programs and 13 universities participating in this initiative. Since its launch in 2018, 351 scholarships have been awarded thanks to funding from the Province of Ontario. 

Would you like to know more about the Vector Institute? 
